# NLP - Text Generation

In [25]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

## Import libraries

In [3]:
import tensorflow as tf

In [4]:
import keras

In [5]:
from tensorflow .keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np

## Load the Corpus

In [7]:
data = open('beatles.txt').read()

In [8]:
corpus = data.lower().split('\n')

In [30]:
corpus

['yesterday, all my troubles seemed so far away',
 "now it looks as though they're here to stay",
 "oh, i believe in yesterday suddenly, i'm not half the man i used to be",
 "there's a shadow hanging over me.",
 "oh, yesterday came suddenly why she had to go i don't know she wouldn't say",
 'i said something wrong, now i long for yesterday yesterday, love was such an easy game to play',
 'now i need a place to hide away',
 "oh, i believe in yesterday why she had to go i don't know she wouldn't say",
 'i said something wrong, now i long for yesterday yesterday, love was such an easy game to play',
 'now i need a place to hide away',
 'oh, i believe in yesterday',
 'mm mm mm mm mm mm mm when i find myself in times of trouble, mother mary comes to me',
 'speaking words of wisdom, let it be',
 'and in my hour of darkness she is standing right in front of me',
 'speaking words of wisdom, let it be',
 'let it be, let it be, let it be, let it be',
 'whisper words of wisdom, let it be and when

## Tokenization

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [10]:
print(tokenizer.word_index)

{'you': 1, 'i': 2, 'the': 3, 'to': 4, 'and': 5, 'love': 6, 'a': 7, 'me': 8, 'in': 9, 'all': 10, 'nah': 11, 'my': 12, 'it': 13, 'know': 14, 'be': 15, 'that': 16, 'of': 17, 'on': 18, 'she': 19, 'is': 20, 'say': 21, 'now': 22, 'oh': 23, 'yeah': 24, 'your': 25, 'get': 26, "don't": 27, 'back': 28, 'let': 29, 'with': 30, 'her': 31, 'can': 32, "it's": 33, 'do': 34, "i'm": 35, 'when': 36, 'come': 37, 'will': 38, 'he': 39, 'but': 40, 'for': 41, 'so': 42, 'need': 43, 'there': 44, 'like': 45, 'no': 46, 'together': 47, "i'll": 48, 'see': 49, 'was': 50, 'never': 51, 'we': 52, 'down': 53, 'way': 54, 'if': 55, 'want': 56, 'out': 57, 'hey': 58, 'one': 59, 'got': 60, 'go': 61, 'well': 62, "can't": 63, 'girl': 64, 'good': 65, 'his': 66, 'sun': 67, 'they': 68, 'home': 69, 'hello': 70, 'day': 71, 'are': 72, 'ah': 73, 'right': 74, 'little': 75, 'at': 76, 'from': 77, 'man': 78, 'here': 79, 'help': 80, 'gonna': 81, 'have': 82, 'not': 83, 'world': 84, 'going': 85, 'what': 86, 'still': 87, 'how': 88, 'too': 89

In [11]:
print(len(tokenizer.word_index))

1628


In [12]:
print(total_words)

1629


## Generating n-gram sequences

In [15]:
input_sequences = []
for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[: i+1]
    input_sequences.append(n_gram_sequence)

input_sequences

[[200, 10],
 [200, 10, 12],
 [200, 10, 12, 907],
 [200, 10, 12, 907, 908],
 [200, 10, 12, 907, 908, 42],
 [200, 10, 12, 907, 908, 42, 909],
 [200, 10, 12, 907, 908, 42, 909, 121],
 [22, 13],
 [22, 13, 638],
 [22, 13, 638, 91],
 [22, 13, 638, 91, 217],
 [22, 13, 638, 91, 217, 349],
 [22, 13, 638, 91, 217, 349, 79],
 [22, 13, 638, 91, 217, 349, 79, 4],
 [22, 13, 638, 91, 217, 349, 79, 4, 314],
 [23, 2],
 [23, 2, 160],
 [23, 2, 160, 9],
 [23, 2, 160, 9, 200],
 [23, 2, 160, 9, 200, 520],
 [23, 2, 160, 9, 200, 520, 35],
 [23, 2, 160, 9, 200, 520, 35, 83],
 [23, 2, 160, 9, 200, 520, 35, 83, 910],
 [23, 2, 160, 9, 200, 520, 35, 83, 910, 3],
 [23, 2, 160, 9, 200, 520, 35, 83, 910, 3, 78],
 [23, 2, 160, 9, 200, 520, 35, 83, 910, 3, 78, 2],
 [23, 2, 160, 9, 200, 520, 35, 83, 910, 3, 78, 2, 911],
 [23, 2, 160, 9, 200, 520, 35, 83, 910, 3, 78, 2, 911, 4],
 [23, 2, 160, 9, 200, 520, 35, 83, 910, 3, 78, 2, 911, 4, 15],
 [173, 7],
 [173, 7, 912],
 [173, 7, 912, 913],
 [173, 7, 912, 913, 397],
 [173, 

## Padding

In [18]:
# Padding

max_sequence_len = max([len(x) for x in input_sequences]) # list comprehension
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')
input_sequences

array([[  0,   0,   0, ...,   0, 200,  10],
       [  0,   0,   0, ..., 200,  10,  12],
       [  0,   0,   0, ...,  10,  12, 907],
       ...,
       [  0,   0,   0, ...,  41,  46,  59],
       [  0,   0,   0, ...,  46,  59,  40],
       [  0,   0,   0, ...,  59,  40,   8]])

In [19]:
max_sequence_len

34

## Model - Bidirectional LSTM

In [20]:
xs, label = input_sequences[:,:-1], input_sequences[:,-1]

In [21]:
label[55]

200

In [22]:
ys = tf.keras.utils.to_categorical(label, num_classes=total_words)
ys[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [23]:
len(ys[0])

1629

In [26]:
model= Sequential()
model.add(Embedding(total_words, 100, input_length= max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words,activation='softmax'))
adam= Adam(lr=0.01)
model.compile(loss='categorical_crossentropy',optimizer=adam, metrics=['accuracy'])
history=model.fit(xs , ys, epochs= 25, verbose=1)
print(model)

Epoch 1/25
392/392 [==============================] - 40s 87ms/step - loss: 6.0349 - accuracy: 0.0542
Epoch 2/25
392/392 [==============================] - 38s 97ms/step - loss: 5.3191 - accuracy: 0.1028
Epoch 3/25
392/392 [==============================] - 37s 94ms/step - loss: 4.7066 - accuracy: 0.1651
Epoch 4/25
392/392 [==============================] - 36s 93ms/step - loss: 4.1452 - accuracy: 0.2355
Epoch 5/25
392/392 [==============================] - 37s 94ms/step - loss: 3.6528 - accuracy: 0.2979
Epoch 6/25
392/392 [==============================] - 37s 94ms/step - loss: 3.2230 - accuracy: 0.3604
Epoch 7/25
392/392 [==============================] - 37s 93ms/step - loss: 2.8512 - accuracy: 0.4225
Epoch 8/25
392/392 [==============================] - 37s 94ms/step - loss: 2.5275 - accuracy: 0.4752
Epoch 9/25
392/392 [==============================] - 38s 97ms/step - loss: 2.2531 - accuracy: 0.5275
Epoch 10/25
392/392 [==============================] - 49s 124ms/step - loss: 2.00

## Text Generation

In [28]:
seed_text = 'The battle in'
next_words = 20

for _ in range(next_words):
  token_list = tokenizer.texts_to_sequences([seed_text])[0]
  token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
  predicted = np.argmax(model.predict(token_list), axis=-1)
  output_words = ""
  for word, index in tokenizer.word_index.items():
    if index==predicted:
      output_word=word
      break
  seed_text += " " + output_word

  print(seed_text)

1/1 [==============================] - 0s 59ms/step
The battle in with
1/1 [==============================] - 0s 77ms/step
The battle in with the
1/1 [==============================] - 0s 30ms/step
The battle in with the dark
1/1 [==============================] - 0s 40ms/step
The battle in with the dark tryin
1/1 [==============================] - 0s 30ms/step
The battle in with the dark tryin to
1/1 [==============================] - 0s 50ms/step
The battle in with the dark tryin to the
1/1 [==============================] - 0s 52ms/step
The battle in with the dark tryin to the faces
1/1 [==============================] - 0s 89ms/step
The battle in with the dark tryin to the faces blackbird
1/1 [==============================] - 0s 70ms/step
The battle in with the dark tryin to the faces blackbird gone
1/1 [==============================] - 0s 47ms/step
The battle in with the dark tryin to the faces blackbird gone in
1/1 [==============================] - 0s 38ms/step
The battle in w